# Decision Tree

## Import libraries

In [1]:
import numpy as np
import matplotlib.pyplot as plt

print("Libraries imported!")

Libraries imported!


## Model Architecture

In [ ]:
class DecisionTree():

    def __init__(self, max_depth=None):
        self.max_depth = max_depth

    def fit(self, X, y):
        pass
    
    def predict(self, X):
        pass

    def _gini(self, y):
        pass